# Classification on text features

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Примеры пайплайнов экстракции признаков

### Обучение

* Изменения в первом этапе, внесенные после защиты:
  * гиперпараметры logreg: C: `[.01, .1, .2, .5, .8, 1., 3, 5, 10]`
  * logreg все еще только с l2 реуляризацией, так как иначе требуется отличный от liblinear солвер, который работает на несколько порядков дольше
  * добавлена модель svm. гиперпараметры: C: `[.1, .5, 1, 3, 5]`, penalty: `['l1', 'l2']`
  * ГП для XGBoost, RandomForest: max_depth: `[3, 5, 10]`, n_estimators: `[500, 1000, 2000]`
  * CatBoost обучается на порядок дольше чем xgb/rf, поэтому обучение не проводилось на нем. GPU версия не работает с разреженными матрицами, вместо этого конвертирует их в dense формат, что не умещается в память

### Результаты

Ниже преведена таблица с результатами. Красным цветом выделен максимум в каждом столбике. Таблица отсортирована по убыванию f1 на тестовом множестве. Из нее можно сделать сделующие выводы:
* Хеширование n-грам в среднем дает лучшие результаты, потому что оно вообще не выбрасывает n-граммы, а использует все (возможно с коллизиями)
* BPE токенизация дала очень сильный буст качества работы алгоритма - все модели с bpe строго лучше остальных (исключение - random forest)
* Ожидаемо, лучшей моделью в среднем оказался xgboost, от него не сильно отстает logreg, random forest оказался хуже
* В принципе лучшей моделью оказалась связка BPE + TfIdf + LogReg, она дала 0.724 на тестовом множестве

Все вычисления производились на сервере с 8-ядерным процессором Ryzen 7 3700X @ 3.6GHz, и заняли ~60 часов

In [5]:
results_df = pd.read_csv('training_results.csv')
results_df_svm = pd.read_csv('training_results_svm.csv')
results_df_lr = pd.read_csv('training_results_slr.csv')
results_df = pd.concat([results_df, results_df_svm, results_df_lr])

In [6]:
results_df = results_df.set_index(['model', 'tokenizer', 'postprocessor', 'vectorizer'])
results_df = results_df.sort_values('test_f1', ascending=False)
def highlight_max(s):
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]
results_df.style.apply(highlight_max)

In [7]:
results_df.groupby('vectorizer').mean().sort_values('test_f1', ascending=False)

,train_f1,test_f1,train_acc,test_acc,train_precision,test_precision,train_recall,test_recall,train_roc_auc,test_roc_auc
vectorizer,,,,,,,,,,
tfidf,0.712078,0.680509,0.732610,0.702024,0.764545,0.731671,0.669656,0.639148,0.732580,0.702131
bow,0.703026,0.677390,0.728053,0.703266,0.766409,0.739568,0.653305,0.628645,0.728018,0.703393
hashing,0.718173,0.673929,0.735708,0.692823,0.764048,0.718212,0.680134,0.637216,0.735682,0.692917
glove_emb,0.633430,0.621858,0.628156,0.616098,0.624925,0.613976,0.642268,0.630027,0.628163,0.616074


In [8]:
results_df.groupby('postprocessor').mean().sort_values('test_f1', ascending=False)

,train_f1,test_f1,train_acc,test_acc,train_precision,test_precision,train_recall,test_recall,train_roc_auc,test_roc_auc
postprocessor,,,,,,,,,,
none,0.742533,0.698972,0.758486,0.716106,0.788203,0.742869,0.704023,0.661867,0.758460,0.716198
stem,0.697794,0.668983,0.721163,0.693258,0.755901,0.725703,0.651885,0.624201,0.721131,0.693376
lemma,0.678071,0.653370,0.694580,0.670586,0.719405,0.694154,0.645957,0.621713,0.694557,0.670669


In [9]:
results_df.groupby('tokenizer').mean().sort_values('test_f1', ascending=False)

,train_f1,test_f1,train_acc,test_acc,train_precision,test_precision,train_recall,test_recall,train_roc_auc,test_roc_auc
tokenizer,,,,,,,,,,
bpe,0.742533,0.698972,0.758486,0.716106,0.788203,0.742869,0.704023,0.661867,0.758460,0.716198
base,0.686523,0.660061,0.705973,0.680303,0.735046,0.707675,0.648498,0.622779,0.705946,0.680400


In [10]:
results_df.groupby('model').mean().sort_values('test_f1', ascending=False)

,train_f1,test_f1,train_acc,test_acc,train_precision,test_precision,train_recall,test_recall,train_roc_auc,test_roc_auc
model,,,,,,,,,,
xgb,0.796378,0.697528,0.805226,0.710400,0.835513,0.732305,0.761124,0.666246,0.805205,0.710475
logreg,0.703931,0.687026,0.713679,0.696628,0.730526,0.712677,0.679610,0.663552,0.713663,0.696685
svm,0.703318,0.684987,0.714048,0.696070,0.732826,0.714783,0.676623,0.658101,0.714026,0.696151
sparse_logreg,0.702805,0.683549,0.712822,0.694734,0.730263,0.712513,0.677771,0.657297,0.712810,0.694779
rf,0.610199,0.605582,0.662859,0.657385,0.725837,0.718887,0.530648,0.527332,0.662797,0.657608


In [25]:
results_df.to_csv('results.csv')